In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.special import inv_boxcox
import catboost as cb
from tqdm import tqdm
from prophet import Prophet
import datetime
import holidays
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

In [2]:
def wape(y_true: np.array, y_pred: np.array):
    return np.sum(np.abs(y_true-y_pred))/np.sum(np.abs(y_true))

In [50]:
train = pd.read_csv("train_preprocessed_w_features.csv")
train['day'] = pd.to_datetime(train['day'], format="%Y-%m-%d")
test = pd.read_csv("test_preprocessed_w_features.csv")
test['day'] = pd.to_datetime(test['day'], format="%Y-%m-%d")

In [51]:
train['month'] = pd.DatetimeIndex(train['day']).month
train['year'] = pd.DatetimeIndex(train['day']).year
test['month'] = pd.DatetimeIndex(test['day']).month
test['year'] = pd.DatetimeIndex(test['day']).year

In [6]:
train['year'].value_counts()

2019    6971552
2020    6831757
2018    4609034
2021    3280617
Name: year, dtype: int64

In [7]:
train['price'] = train['selling_price']/train['num_sales']
train['sell_in'] = train['supplier_price']/train['num_sales']

In [9]:
stat_month = train.groupby(["year", "month", "sku"]).agg({"check_id": "count"}).reset_index().sort_values(by='check_id')
stat_month.rename(columns={"check_id": "count"}, inplace=True)
stat_month = stat_month.groupby("sku").agg({"count": "mean"})
stat_month = stat_month.reset_index()

train = train.merge(stat_month, on='sku', how='left')

## CatBoost

In [19]:
train_df = train.groupby(["day", "shop_id", "sku"]).agg({"price": "mean", "num_sales": "sum", 
                                                         "count": "mean", "sell_in": "mean", "month": "first",
                                                         "promo_id": "first", "discount": "mean"}).reset_index()


In [21]:
train_df['target'] = train_df['num_sales']
train_df.drop(["num_sales"], axis=1, inplace=True)

In [23]:
train_df['id'] = range(1, len(train_df)+1)

In [ ]:
today_date = train_df['day'].max() - datetime.timedelta(days=362)
today_date

Timestamp('2020-07-03 00:00:00')

In [27]:
y_train = train_df['target']
X_train = train_df.drop(["target", "day", "shop_id", "sku"], axis=1)

# y_train = train_df[train_df["day"] < today_date]['target']
# X_train = train_df[train_df["day"] < today_date].drop(["target", "day", "shop_id", "sku"], axis=1)

# y_test = train_df[train_df["day"] >= today_date]['target']
# X_test = train_df[train_df["day"] >= today_date].drop(["target", "day", "shop_id", "sku"], axis=1)

In [32]:
model = cb.CatBoostRegressor(loss_function='RMSE')

In [33]:
model.fit(X_train.drop(['id'], axis=1), y_train, cat_features=['promo_id'])

Learning rate set to 0.192555
0:	learn: 3.4371361	total: 4.89s	remaining: 1h 21m 30s
1:	learn: 3.3260763	total: 9.22s	remaining: 1h 16m 40s
2:	learn: 3.2481080	total: 12.6s	remaining: 1h 10m 2s
3:	learn: 3.1921174	total: 16.7s	remaining: 1h 9m 14s
4:	learn: 3.1502318	total: 19.6s	remaining: 1h 5m 3s
5:	learn: 3.1107871	total: 22.7s	remaining: 1h 2m 33s
6:	learn: 3.0180564	total: 25.5s	remaining: 1h 14s
7:	learn: 2.9500063	total: 28.5s	remaining: 58m 53s
8:	learn: 2.9174018	total: 32.4s	remaining: 59m 31s
9:	learn: 2.8267594	total: 35.3s	remaining: 58m 9s
10:	learn: 2.7703256	total: 37.2s	remaining: 55m 48s
11:	learn: 2.7087650	total: 40.1s	remaining: 54m 57s
12:	learn: 2.6548811	total: 42.5s	remaining: 53m 48s
13:	learn: 2.6412144	total: 44.7s	remaining: 52m 27s
14:	learn: 2.6163330	total: 47s	remaining: 51m 26s
15:	learn: 2.5932188	total: 49.7s	remaining: 50m 54s
16:	learn: 2.5603864	total: 52.5s	remaining: 50m 34s
17:	learn: 2.5404893	total: 55s	remaining: 50m 1s
18:	learn: 2.5227089

In [158]:
#wape(y_test, pred)

0.3664232042203915

In [35]:
importances = list()
importances.append(pd.DataFrame({'feature_importance': model.get_feature_importance(), 
          'feature_names': X_train.drop('id', axis=1).columns}).sort_values(by=['feature_importance'], 
                                                       ascending=False))

In [36]:
importances

[   feature_importance feature_names
 5           32.653720      discount
 2           19.053769       sell_in
 0           18.603446         price
 1           14.675167         count
 4           11.628519      promo_id
 3            3.385378         month]

In [34]:
model.save_model("catboost_final", format="cbm")

In [166]:
# m = cb.CatBoostRegressor() 
# m.load_model("catboost_final")

## Needed feaatures for test set

In [182]:
test_prices = pd.read_csv("test_new.csv", index_col=0)
test_prices['day'] = pd.to_datetime(test_prices['day'], format="%Y-%m-%d")

/Users/thingavu/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



In [187]:
test_prices['month'] = pd.DatetimeIndex(test_prices['day']).month
test_prices['year'] = pd.DatetimeIndex(test_prices['day']).year

In [188]:
test_prices.shape, test_prices['sku'].nunique()

((2666084, 28), 55001)

In [189]:
test_prices.head()

,day,shop_id,check_id,time,sku,promo_id,supplier_price,selling_price,discount,class,...,type_format_id,type_wealth_id,is_store,is_active,is_ex_billa,num,price,sell_in,month,year
0,2021-10-25,23,10,22:25:37,114519,300187,64.84,86.26,28.72,123369,...,1,1,1,1,0,1.0,86.26,64.84,10,2021
1,2021-10-25,23,11,21:25:01,4405,285514,8.80,0.00,51.59,123371,...,1,1,1,1,0,1.0,0.00,8.80,10,2021
2,2021-10-25,23,11,21:25:01,99020,296267,384.60,599.00,347.39,123371,...,1,1,1,1,0,1.0,599.00,384.60,10,2021
3,2021-10-25,23,10,22:25:37,88039,299802,28.68,41.27,23.95,123369,...,1,1,1,1,0,1.0,41.27,28.68,10,2021
4,2021-10-25,23,10,22:25:37,79182,51588,1.47,5.49,0.30,123371,...,1,1,1,1,0,1.0,5.49,1.47,10,2021


In [190]:
stat_month = test_prices.groupby(["year", "month", "sku"]).agg({"check_id": "count"}).reset_index().sort_values(by='check_id')
stat_month.rename(columns={"check_id": "count"}, inplace=True)
stat_month = stat_month.groupby("sku").agg({"count": "mean"})
stat_month = stat_month.reset_index()

test_prices = test_prices.merge(stat_month, on='sku', how='left')
test_prices

,day,shop_id,check_id,time,sku,promo_id,supplier_price,selling_price,discount,class,...,type_wealth_id,is_store,is_active,is_ex_billa,num,price,sell_in,month,year,count
0,2021-10-25,23,10,22:25:37,114519,300187,64.84,86.26,28.72,123369,...,1,1,1,0,1.0,86.260000,64.840000,10,2021,541.000000
1,2021-10-25,23,11,21:25:01,4405,285514,8.80,0.00,51.59,123371,...,1,1,1,0,1.0,0.000000,8.800000,10,2021,10092.666667
2,2021-10-25,23,11,21:25:01,99020,296267,384.60,599.00,347.39,123371,...,1,1,1,0,1.0,599.000000,384.600000,10,2021,5.200000
3,2021-10-25,23,10,22:25:37,88039,299802,28.68,41.27,23.95,123369,...,1,1,1,0,1.0,41.270000,28.680000,10,2021,1033.000000
4,2021-10-25,23,10,22:25:37,79182,51588,1.47,5.49,0.30,123371,...,1,1,1,0,1.0,5.490000,1.470000,10,2021,12925.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666079,2021-10-23,42,2666084,10:52:58,70,51588,688.37,986.45,51.92,123369,...,2,1,1,0,3.0,328.816667,229.456667,10,2021,28.400000
2666080,2021-10-23,42,2666084,10:52:58,94981,286290,85.83,99.99,109.50,123371,...,2,1,1,0,1.0,99.990000,85.830000,10,2021,2.000000
2666081,2021-10-23,42,2666084,10:52:58,96905,51588,7.78,13.50,0.72,123369,...,2,1,1,0,1.0,13.500000,7.780000,10,2021,62.000000
2666082,2021-10-23,42,2666084,10:52:58,4405,285514,8.80,0.00,51.59,123371,...,2,1,1,0,1.0,0.000000,8.800000,10,2021,10092.666667


In [198]:
test_prices['promo_id'] = test_prices['promo_id'].astype(int)

test_df = test_prices.groupby(["day", "shop_id", "sku"]).agg({"price": "mean", "count": "mean", "sell_in": "mean", "month": "first",
                                                         "promo_id": "first", "discount": "mean"}).reset_index()

In [199]:
test_df['id'] = range(1, len(test_df)+1)

In [200]:
predictions = model.predict(test_df[['price', 'count', 'sell_in', 'month', 'promo_id', 'discount']])

In [201]:
test_df['prediction'] = predictions

In [202]:
test_df

,day,shop_id,sku,price,count,sell_in,month,promo_id,discount,id,prediction
0,2021-07-01,0,17,95.7200,332.60,68.0900,7,225972,16.89,1,1.572795
1,2021-07-01,0,49,61.5400,229.60,17.5300,7,231968,10.86,2,0.909536
2,2021-07-01,0,79,35.2700,180.40,22.7700,7,225972,6.22,3,1.795373
3,2021-07-01,0,109,225.2275,1168.80,213.2225,7,225972,79.50,4,3.341803
4,2021-07-01,0,213,32.4900,55.80,13.8300,7,51588,1.80,5,1.166016
...,...,...,...,...,...,...,...,...,...,...,...
2253775,2021-11-30,59,118891,329.8900,96.75,247.1600,11,319075,514.80,2253776,2.696585
2253776,2021-11-30,59,118943,32.5900,198.40,25.5200,11,51588,3.60,2253777,2.642239
2253777,2021-11-30,59,119087,39.8900,32.80,16.8800,11,302147,22.30,2253778,1.577934
2253778,2021-11-30,59,119442,199.9900,18.80,127.6000,11,306940,221.10,2253779,1.498251


## Submission

In [203]:
submission = test_df[['day', 'shop_id', 'sku', 'prediction']]
submission

,day,shop_id,sku,prediction
0,2021-07-01,0,17,1.572795
1,2021-07-01,0,49,0.909536
2,2021-07-01,0,79,1.795373
3,2021-07-01,0,109,3.341803
4,2021-07-01,0,213,1.166016
...,...,...,...,...
2253775,2021-11-30,59,118891,2.696585
2253776,2021-11-30,59,118943,2.642239
2253777,2021-11-30,59,119087,1.577934
2253778,2021-11-30,59,119442,1.498251


In [204]:
submission['day'] = submission['day'].dt.strftime('%Y%m%d')
submission

/var/folders/64/svd0gdmx7kz_3vg0x7752y440000gn/T/ipykernel_83582/3934297038.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,day,shop_id,sku,prediction
0,20210701,0,17,1.572795
1,20210701,0,49,0.909536
2,20210701,0,79,1.795373
3,20210701,0,109,3.341803
4,20210701,0,213,1.166016
...,...,...,...,...
2253775,20211130,59,118891,2.696585
2253776,20211130,59,118943,2.642239
2253777,20211130,59,119087,1.577934
2253778,20211130,59,119442,1.498251


In [208]:
shop_id = pd.read_csv("shop_id.csv")
sku = pd.read_csv("sku.csv")
shop_id = shop_id.to_dict()['index']
sku = sku.to_dict()['index']

submission['shop_id'] = submission['shop_id'].map(shop_id)
submission['sku'] = submission['sku'].map(sku)

submission = submission.rename(columns={'prediction': 'demand'})

In [209]:
submission

,day,shop_id,sku,demand
0,20210701,d04feef70623eaf7167f9172da8650db,6c7376da8563e2a1c4a554dd3623f832,1.572795
1,20210701,d04feef70623eaf7167f9172da8650db,23f552a9126cedd94df4a760107773de,0.909536
2,20210701,d04feef70623eaf7167f9172da8650db,3b70da5574ddb629f9fe6440432894f0,1.795373
3,20210701,d04feef70623eaf7167f9172da8650db,ef7bf412117548cb80b3f3b33a12977d,3.341803
4,20210701,d04feef70623eaf7167f9172da8650db,acbea6afda31885cb996df4569c0f650,1.166016
...,...,...,...,...
2253775,20211130,7d7c45b9a935cf9d845fc75679a41559,db26c36d69925ebe6d6745beb3d5e0df,2.696585
2253776,20211130,7d7c45b9a935cf9d845fc75679a41559,d36f89e079d802acade5e4746523ef5c,2.642239
2253777,20211130,7d7c45b9a935cf9d845fc75679a41559,eeb335affeb679d160d1afe7393d6ce7,1.577934
2253778,20211130,7d7c45b9a935cf9d845fc75679a41559,fba66841370ea48ab47c027bbacc18f1,1.498251


In [211]:
submission.to_csv('pricing_hackathon_submission.csv')

In [73]:
mean_prices = train.groupby(['shop_id', 'sku']).agg({"price": "mean"}).reset_index()

In [77]:
test = test.merge(mean_prices, on=['shop_id', 'sku'], how='left')
test

,day,shop_id,check_id,time,sku,promo_id,supplier_price,selling_price,discount,class,...,type_loc_id,type_size_id,type_format_id,type_wealth_id,is_store,is_active,is_ex_billa,month,year,price
0,2021-10-25,23,10,22:25:37,114519,300187,64.84,86.26,28.72,123369,...,1,8,1,1,1,1,0,10,2021,33.990569
1,2021-10-25,23,11,21:25:01,4405,285514,8.80,0.00,51.59,123371,...,1,8,1,1,1,1,0,10,2021,NaN
2,2021-10-25,23,11,21:25:01,99020,296267,384.60,599.00,347.39,123371,...,1,8,1,1,1,1,0,10,2021,NaN
3,2021-10-25,23,10,22:25:37,88039,299802,28.68,41.27,23.95,123369,...,1,8,1,1,1,1,0,10,2021,29.846492
4,2021-10-25,23,10,22:25:37,79182,51588,1.47,5.49,0.30,123371,...,1,8,1,1,1,1,0,10,2021,3.555905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666079,2021-10-23,42,2666084,10:52:58,70,51588,688.37,986.45,51.92,123369,...,2,4,1,2,1,1,0,10,2021,127.664313
2666080,2021-10-23,42,2666084,10:52:58,94981,286290,85.83,99.99,109.50,123371,...,2,4,1,2,1,1,0,10,2021,NaN
2666081,2021-10-23,42,2666084,10:52:58,96905,51588,7.78,13.50,0.72,123369,...,2,4,1,2,1,1,0,10,2021,41.355708
2666082,2021-10-23,42,2666084,10:52:58,4405,285514,8.80,0.00,51.59,123371,...,2,4,1,2,1,1,0,10,2021,NaN


In [101]:
test['sku'].nunique()

55001

In [102]:
test_w_prices = test[~test['price'].isna()]

In [80]:
sku_to_est = test[test['price'].isna()]['sku'].unique()
sku_to_est

array([  4405,  99020,  25367, ...,  42684,  47855, 101040])

In [82]:
hierarchy = pd.read_csv("hierarchy_preprocessed.csv")

In [86]:
subcategories_to_est = hierarchy[hierarchy['sku'].isin(sku_to_est)]['subcategory'].unique()

In [89]:
subcategory_prices = train[train['subcategory'].isin(subcategories_to_est)].groupby('subcategory').agg({"price": "mean"}).reset_index()

In [95]:
subcategory_prices['subcategory'].nunique(), subcategory_prices.shape

(1977, (1977, 2))

In [97]:
hierarchy[hierarchy['sku'].isin(sku_to_est)].shape

(42584, 5)

In [99]:
additional_prices = hierarchy[hierarchy['sku'].isin(sku_to_est)].merge(subcategory_prices, on='subcategory', how='left')
additional_prices

,sku,class,group,category,subcategory,price
0,0,123369,123369,122673,121845,117.394454
1,2,123371,123371,122367,102400,262.319205
2,4,123369,123366,121283,121283,164.379655
3,8,123369,123300,123057,123057,104.274022
4,9,123369,123369,122951,122951,169.421682
...,...,...,...,...,...,...
42579,123303,123369,123369,123303,123303,229.061852
42580,123305,123369,123366,123305,123305,36.154630
42581,123315,123371,123371,123315,123315,137.801390
42582,123337,123369,123366,123337,123337,149.890908


In [103]:
test[test['price'].isna()].shape

(344104, 26)

In [106]:
test = test[test['price'].isna()].drop('price', axis=1).merge(additional_prices[['sku', 'price']], on=['sku'], how='left')
test

,day,shop_id,check_id,time,sku,promo_id,supplier_price,selling_price,discount,class,...,type_loc_id,type_size_id,type_format_id,type_wealth_id,is_store,is_active,is_ex_billa,month,year,price
0,2021-10-25,23,11,21:25:01,4405,285514,8.80,0.00,51.59,123371,...,1,8,1,1,1,1,0,10,2021,2.854609
1,2021-10-25,23,11,21:25:01,99020,296267,384.60,599.00,347.39,123371,...,1,8,1,1,1,1,0,10,2021,929.208941
2,2021-10-25,23,10,22:25:37,25367,285093,1659.65,1449.11,883.58,123371,...,1,8,1,1,1,1,0,10,2021,1070.820804
3,2021-10-25,23,10,22:25:37,4405,285514,8.80,0.00,51.59,123371,...,1,8,1,1,1,1,0,10,2021,2.854609
4,2021-10-25,23,11,21:25:01,50371,285093,1627.86,1449.99,882.70,123371,...,1,8,1,1,1,1,0,10,2021,1070.820804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344099,2021-10-23,42,2666056,21:33:34,4405,285514,17.60,0.00,103.18,123371,...,2,4,1,2,1,1,0,10,2021,2.854609
344100,2021-10-23,42,2666076,21:03:50,4405,285514,8.80,0.00,51.59,123371,...,2,4,1,2,1,1,0,10,2021,2.854609
344101,2021-10-23,42,2666084,10:52:58,94981,286290,85.83,99.99,109.50,123371,...,2,4,1,2,1,1,0,10,2021,81.075409
344102,2021-10-23,42,2666084,10:52:58,4405,285514,8.80,0.00,51.59,123371,...,2,4,1,2,1,1,0,10,2021,2.854609


In [108]:
test_w_prices = pd.concat([test_w_prices, test[~test['price'].isna()]])

In [114]:
test[test['price'].isna()]['sku'].unique()

array([  1612,  42536,  91680,  12692, 105074,  52158,  68024,  26080,
        51611,  24747,  28767,  65557,  59480, 121920, 114116,  74583,
        70336,  43167, 109456,  30296,  26079,  21260,  29953, 113511,
        17235,  70416])

In [112]:
sku_to_est = test[test['price'].isna()]['sku'].unique()
categories_to_est = hierarchy[hierarchy['sku'].isin(sku_to_est)]['category'].unique()
category_prices = train[train['category'].isin(categories_to_est)].groupby('category').agg({"price": "mean"}).reset_index()
additional_prices = hierarchy[hierarchy['sku'].isin(sku_to_est)].merge(category_prices, on='category', how='left')
additional_prices

,sku,class,group,category,subcategory,price
0,1612,123369,123366,116090,21260,92.040435
1,12692,123369,123366,122916,114116,126.032558
2,17235,123369,123366,121920,121920,33.587331
3,21260,123369,123366,116090,21260,92.040435
4,24747,123369,123366,121920,121920,33.587331
5,26079,123369,123366,122916,114116,126.032558
6,26080,123369,123366,122916,114116,126.032558
7,28767,123369,123366,122916,114116,126.032558
8,29953,123369,123366,122916,114116,126.032558
9,30296,123369,123366,122916,114116,126.032558


In [113]:
additional_prices[additional_prices['sku'].isin(sku_to_est)]

,sku,class,group,category,subcategory,price
0,1612,123369,123366,116090,21260,92.040435
1,12692,123369,123366,122916,114116,126.032558
2,17235,123369,123366,121920,121920,33.587331
3,21260,123369,123366,116090,21260,92.040435
4,24747,123369,123366,121920,121920,33.587331
5,26079,123369,123366,122916,114116,126.032558
6,26080,123369,123366,122916,114116,126.032558
7,28767,123369,123366,122916,114116,126.032558
8,29953,123369,123366,122916,114116,126.032558
9,30296,123369,123366,122916,114116,126.032558


In [116]:
test = test[test['price'].isna()].drop('price', axis=1).merge(additional_prices[['sku', 'price']], on=['sku'], how='left')

In [119]:
test_w_prices = pd.concat([test_w_prices, test])

In [120]:
test_w_prices.shape

(2666084, 26)

In [123]:
test_w_prices['est_num_sales'] = test_w_prices['selling_price']/test_w_prices['price']
test_w_prices

,day,shop_id,check_id,time,sku,promo_id,supplier_price,selling_price,discount,class,...,type_size_id,type_format_id,type_wealth_id,is_store,is_active,is_ex_billa,month,year,price,est_num_sales
0,2021-10-25,23,10,22:25:37,114519,300187,64.84,86.26,28.72,123369,...,8,1,1,1,1,0,10,2021,33.990569,2.537763
3,2021-10-25,23,10,22:25:37,88039,299802,28.68,41.27,23.95,123369,...,8,1,1,1,1,0,10,2021,29.846492,1.382742
4,2021-10-25,23,10,22:25:37,79182,51588,1.47,5.49,0.30,123371,...,8,1,1,1,1,0,10,2021,3.555905,1.543911
6,2021-10-25,23,10,22:25:37,13210,51588,27.24,69.99,3.70,123369,...,8,1,1,1,1,0,10,2021,56.653333,1.235408
7,2021-10-25,23,10,22:25:37,105667,299436,31.29,47.89,23.50,123369,...,8,1,1,1,1,0,10,2021,51.025000,0.938560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,2021-10-30,25,2637746,11:43:09,26080,51588,240.22,448.95,23.72,123369,...,3,1,2,1,1,0,10,2021,126.032558,3.562175
76,2021-10-30,25,2637746,11:43:09,105074,51588,38.25,71.28,3.76,123369,...,3,1,2,1,1,0,10,2021,126.032558,0.565568
77,2021-11-04,50,2647122,12:25:23,17235,51588,56.86,101.89,5.40,123369,...,3,1,2,1,1,0,11,2021,33.587331,3.033584
78,2021-10-23,15,2662996,13:03:00,105074,51588,26.21,48.84,2.57,123369,...,3,1,2,1,1,0,10,2021,126.032558,0.387519


In [124]:
stat_month = test_w_prices.groupby(["year", "month", "sku"]).agg({"check_id": "count"}).reset_index().sort_values(by='check_id')
stat_month.rename(columns={"check_id": "count"}, inplace=True)
stat_month = stat_month.groupby("sku").agg({"count": "mean"})
stat_month = stat_month.reset_index()

test_w_prices = test_w_prices.merge(stat_month, on='sku', how='left')
test_w_prices

,day,shop_id,check_id,time,sku,promo_id,supplier_price,selling_price,discount,class,...,type_format_id,type_wealth_id,is_store,is_active,is_ex_billa,month,year,price,est_num_sales,count
0,2021-10-25,23,10,22:25:37,114519,300187,64.84,86.26,28.72,123369,...,1,1,1,1,0,10,2021,33.990569,2.537763,541.0
1,2021-10-25,23,10,22:25:37,88039,299802,28.68,41.27,23.95,123369,...,1,1,1,1,0,10,2021,29.846492,1.382742,1033.0
2,2021-10-25,23,10,22:25:37,79182,51588,1.47,5.49,0.30,123371,...,1,1,1,1,0,10,2021,3.555905,1.543911,12925.8
3,2021-10-25,23,10,22:25:37,13210,51588,27.24,69.99,3.70,123369,...,1,1,1,1,0,10,2021,56.653333,1.235408,6.0
4,2021-10-25,23,10,22:25:37,105667,299436,31.29,47.89,23.50,123369,...,1,1,1,1,0,10,2021,51.025000,0.938560,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666079,2021-10-30,25,2637746,11:43:09,26080,51588,240.22,448.95,23.72,123369,...,1,2,1,1,0,10,2021,126.032558,3.562175,1.5
2666080,2021-10-30,25,2637746,11:43:09,105074,51588,38.25,71.28,3.76,123369,...,1,2,1,1,0,10,2021,126.032558,0.565568,2.5
2666081,2021-11-04,50,2647122,12:25:23,17235,51588,56.86,101.89,5.40,123369,...,1,2,1,1,0,11,2021,33.587331,3.033584,1.0
2666082,2021-10-23,15,2662996,13:03:00,105074,51588,26.21,48.84,2.57,123369,...,1,2,1,1,0,10,2021,126.032558,0.387519,2.5


In [138]:
test_w_prices['sell_in'] = test_w_prices['supplier_price']/test_w_prices['est_num_sales']
test_w_prices['promo_id'] = test_w_prices['promo_id'].astype(int)

test_df = test_w_prices.groupby(["day", "shop_id", "sku"]).agg({"price": "mean", "count": "mean", "sell_in": "mean", "month": "first",
                                                         "promo_id": "first", "discount": "mean"}).reset_index()

In [140]:
X_train.head()

,price,count,sell_in,month,promo_id,discount,id
0,34.990000,19.000000,24.730000,1,30488,12.40,1
1,82.495308,81.600000,55.721249,1,145965,173.22,2
2,18.490000,38.769231,10.500000,1,115458,61.20,3
3,99.772329,1390.976190,95.683012,1,233553,96.68,4
4,142.090000,2.482759,120.360000,1,164866,71.59,5


In [142]:
test_df['id'] = range(1, len(test_df)+1)

In [143]:
predictions = model.predict(test_df[['price', 'count', 'sell_in', 'month', 'promo_id', 'discount']])

In [145]:
test_df['prediction'] = predictions

In [168]:
test_df.to_csv('test_to_predict.csv')

In [149]:
sub = test_df[['day', 'shop_id', 'sku', 'prediction']]
sub

,day,shop_id,sku,prediction
0,2021-07-01,0,17,2.181779
1,2021-07-01,0,49,0.890574
2,2021-07-01,0,79,1.910745
3,2021-07-01,0,109,5.472796
4,2021-07-01,0,213,1.323421
...,...,...,...,...
2253775,2021-11-30,59,118891,2.696585
2253776,2021-11-30,59,118943,2.958249
2253777,2021-11-30,59,119087,1.650848
2253778,2021-11-30,59,119442,1.578079


In [150]:
sub['day'] = sub['day'].dt.strftime('%Y%m%d')

/var/folders/64/svd0gdmx7kz_3vg0x7752y440000gn/T/ipykernel_83582/4082614251.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [153]:
shop_id = pd.read_csv("shop_id.csv")
sku = pd.read_csv("sku.csv")

In [158]:
sku.head()

,index,sku
0,16b4bff70d409ddb77c8107a528cf94b,0
1,fc6867c8b4ce831c16fa7961958270d4,1
2,1b3353cb266de9c2bcc2fc5b911973f0,2
3,a3d57e327b84fb971e09b2b33c890538,3
4,1c71460770f458c06a29ed650c0974cc,4


In [159]:
shop_id = shop_id.to_dict()['index']
sku = sku.to_dict()['index']

In [160]:
sub['shop_id'] = sub['shop_id'].map(shop_id)
sub['sku'] = sub['sku'].map(sku)
sub

/var/folders/64/svd0gdmx7kz_3vg0x7752y440000gn/T/ipykernel_83582/4024308223.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/64/svd0gdmx7kz_3vg0x7752y440000gn/T/ipykernel_83582/4024308223.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,day,shop_id,sku,prediction
0,20210701,d04feef70623eaf7167f9172da8650db,6c7376da8563e2a1c4a554dd3623f832,2.181779
1,20210701,d04feef70623eaf7167f9172da8650db,23f552a9126cedd94df4a760107773de,0.890574
2,20210701,d04feef70623eaf7167f9172da8650db,3b70da5574ddb629f9fe6440432894f0,1.910745
3,20210701,d04feef70623eaf7167f9172da8650db,ef7bf412117548cb80b3f3b33a12977d,5.472796
4,20210701,d04feef70623eaf7167f9172da8650db,acbea6afda31885cb996df4569c0f650,1.323421
...,...,...,...,...
2253775,20211130,7d7c45b9a935cf9d845fc75679a41559,db26c36d69925ebe6d6745beb3d5e0df,2.696585
2253776,20211130,7d7c45b9a935cf9d845fc75679a41559,d36f89e079d802acade5e4746523ef5c,2.958249
2253777,20211130,7d7c45b9a935cf9d845fc75679a41559,eeb335affeb679d160d1afe7393d6ce7,1.650848
2253778,20211130,7d7c45b9a935cf9d845fc75679a41559,fba66841370ea48ab47c027bbacc18f1,1.578079


In [161]:
sub = sub.rename(columns={'prediction': 'demand'})
sub

,day,shop_id,sku,demand
0,20210701,d04feef70623eaf7167f9172da8650db,6c7376da8563e2a1c4a554dd3623f832,2.181779
1,20210701,d04feef70623eaf7167f9172da8650db,23f552a9126cedd94df4a760107773de,0.890574
2,20210701,d04feef70623eaf7167f9172da8650db,3b70da5574ddb629f9fe6440432894f0,1.910745
3,20210701,d04feef70623eaf7167f9172da8650db,ef7bf412117548cb80b3f3b33a12977d,5.472796
4,20210701,d04feef70623eaf7167f9172da8650db,acbea6afda31885cb996df4569c0f650,1.323421
...,...,...,...,...
2253775,20211130,7d7c45b9a935cf9d845fc75679a41559,db26c36d69925ebe6d6745beb3d5e0df,2.696585
2253776,20211130,7d7c45b9a935cf9d845fc75679a41559,d36f89e079d802acade5e4746523ef5c,2.958249
2253777,20211130,7d7c45b9a935cf9d845fc75679a41559,eeb335affeb679d160d1afe7393d6ce7,1.650848
2253778,20211130,7d7c45b9a935cf9d845fc75679a41559,fba66841370ea48ab47c027bbacc18f1,1.578079


In [162]:
sub.isna().sum()

day        0
shop_id    0
sku        0
demand     0
dtype: int64

In [163]:
sub.to_csv('pricing_hackathon_submission.csv')